In [1]:
import sys
sys.path.append("../common_code/")
import common as com

C++ is very close to C, so there's not much of a surprise that it can be used with the elevation mechanism. We can use it to launch an application in the elevated mode using the scripts we designed for use with C programs. Here we demonstarte a simple C++ program that executes write system calls in a loop and times them.

We start by building the code that will measure the throughput for executing write system calls in a loop.

In [2]:
com.run_cmd("make -C " + com.symbios_path + "/LinuxPrototypes/cpp/ clean")
com.run_cmd("make -C " + com.symbios_path + "/LinuxPrototypes/cpp/")

CompletedProcess(args=['make', '-C', '../..//LinuxPrototypes/cpp/'], returncode=0, stdout="make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'\ng++ -Wall -Wextra -Werror  -o write_loop write_loop.cc\nmake: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'\n", stderr='')

Here we run the program for 1 byte writes and a ~1M writes to a ramdisk. We see a throughtput of around 2 Mb/s on this machine.

In [25]:
ret = com.run_cmd("make -C " + com.symbios_path + "/LinuxPrototypes/cpp/ run")

print(ret.stdout)

throughput_standard = ret.stdout.split("Throughput: ")[1].split(" ")[0]


make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'
./write_loop 1 $((1<<20)) /tmp/test
File /tmp/test exists, deleting
Elapsed time: 0.462632
Throughput: 2.161545 Mb/s
K Writes/sec: 2213.422331 
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'



Next we run the same experiment, no change to the executable, we simply LD_PRELOAD the shortcut library and run the same program.

In [26]:

import os

# Add to path variable
os.environ["PATH"] += os.pathsep + com.symbios_path + "Tools/bin/shortcut"
ret = com.run_cmd(run_sc)

print(ret.stdout)

throughput_sc = ret.stdout.split("Throughput: ")[1].split(" ")[0]


make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'
shortcut.sh -v -be -s 'write->__x64_sys_write' --- ./write_loop 1 $((1<<20)) /tmp/test
Script arguments: -v -be -s write->__x64_sys_write 
APP_AND_ARGS:  ./write_loop 1 1048576 /tmp/test
ELEVATE_FN_LIST: 
LOWER_FN_LIST: 
SHORTCUT_FN_LIST:  write->__x64_sys_write
BEGIN_ELE: 1
ENVT_VARS:  BEGIN_ELE=1 SHORTCUT_write_TO___x64_sys_write=1
RUN_CMD: LD_LIBRARY_PATH=/home/sym/Symbi-OS/Symlib/dynam_build:/home/sym/Symbi-OS/Tools/bin/shortcut/deep_sc  BEGIN_ELE=1 SHORTCUT_write_TO___x64_sys_write=1 LD_PRELOAD=/home/sym/Symbi-OS/Tools/bin/shortcut/sc_lib.so  ./write_loop 1 1048576 /tmp/test 
File /tmp/test exists, deleting
Elapsed time: 0.354197
Throughput: 2.823288 Mb/s
K Writes/sec: 2891.046508 
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/cpp'



Comparing the two results, we can see if the shortcutted c++ program achieves a throughput improvement.

In [31]:
print("Throughput standard: " + throughput_standard)
print("Throughput shortcut: " + throughput_sc)
# reverse the above
print("Speedup: " + str(float(throughput_sc)/float(throughput_standard)))

Throughput standard: 2.161545
Throughput shortcut: 2.823288
Speedup: 1.3061435223416584
